In [1]:
from gana.block.program import Prg
from gana.sets.index import I
from gana.sets.variable import V
from gana.sets.parameter import P
from gana.operations.composition import inf, sup
from gana.sets.theta import T

In [2]:
def p1():
    p1_ = Prg()
    p1_.i = I(size=2)
    p1_.j = I('a', 'b', 'c')
    p1_.x = V(p1_.i)
    p1_.y = V(p1_.j)
    p1_.c1 = sum(p1_.x) <= 12
    p1_.c2 = p1_.y(p1_.j) <= 20
    p1_.o1 = sup(40 * sum(p1_.x) + 30 * sum(p1_.y))
    return p1_


def p2():
    p2_ = Prg()
    p2_.k = I(size=2)
    p2_.z = V(p2_.k)
    p2_.c3 = sum(p2_.z) >= 4
    p2_.o2 = inf(3 * sum(p2_.z))
    return p2_

In [3]:
p1 = p1()
p2 = p2()

In [4]:
p = p1 + p2

In [6]:
p.c3.one.two.n

0

In [7]:
p.C

[[-40.0, -40.0, -30.0, -30.0, -30.0, 0, 0], [3.0, 3.0, 0, 0, 0, 0, 0]]

In [2]:
def p():
    p_ = Prg()
    p_.y = I(size=1)
    p_.q = I(size=3)
    p_.res_cons = I('solar')
    p_.res_dem = I('power')
    p_.res_stg = I('charge')
    p_.res = p_.res_cons | p_.res_dem | p_.res_stg
    p_.pro_var = I('pv')
    p_.pro_cer = I('li', 'li_d')
    p_.pro = p_.pro_var | p_.pro_cer
    p_.dm_fac = P(p_.power, p_.q, _=[0.5, 1, 0.5])
    p_.pv_fac = P(p_.pv, p_.q, _=[1, 0, 0.5])
    p_.demand = P(p_.res_dem, p_.q, _=[100] * 3)
    p_.capex = P(p_.pro, p_.y, _=[5000, 1000, 0])
    p_.fopex = P(p_.pro, p_.y, _=[500, 100, 0])
    p_.vopex = P(p_.pro, p_.y, _=[10, 50, 0])
    p_.cap_p = V(p_.pro, p_.y)
    p_.cap_s = V(p_.res_stg, p_.y)
    p_.sell = V(p_.res_dem, p_.q)
    p_.con = V(p_.res_cons, p_.q)
    p_.inv = V(p_.res_stg, p_.q)
    p_.prod = V(p_.pro, p_.q)
    p_.ex_cap = V(p_.pro, p_.y)
    p_.ex_fop = V(p_.pro, p_.y)
    p_.ex_vop = V(p_.pro, p_.y)
    p_.con_vopex = p_.ex_vop(p_.pro, p_.y) == p_.vopex(p_.pro, p_.y) * sum(
        p_.prod(p_.pro, q) for q in p_.q
    )
    p_.con_capmax = p_.cap_p(p_.pro, p_.y) <= 200
    p_.con_capstg = p_.cap_s(p_.charge, p_.y) <= 200
    p_.con_consmax = p_.con(p_.res_cons, p_.q) <= 200
    p_.con_sell = p_.sell(p_.power, p_.q) >= p_.dm_fac(p_.power, p_.q) * p_.demand(
        p_.power, p_.q
    )
    p_.con_pv = p_.prod(p_.pv, p_.q) <= p_.pv_fac(p_.pv, p_.q) * p_.cap_p(p_.pv, p_.y)
    p_.con_prod = p_.prod(p_.pro_cer, p_.q) <= p_.cap_p(p_.pro_cer, p_.y)
    p_.con_inv = p_.inv(p_.charge, p_.q) <= p_.cap_s(p_.charge, p_.y)
    p_.con_capex = p_.ex_cap(p_.pro, p_.y) == p_.capex(p_.pro, p_.y) * p_.cap_p(
        p_.pro, p_.y
    )
    p_.con_fopex = p_.ex_fop(p_.pro, p_.y) == p_.fopex(p_.pro, p_.y) * p_.cap_p(
        p_.pro, p_.y
    )
    p_.con_solar = p_.prod(p_.pv, p_.q) == p_.con(p_.solar, p_.q)
    p_.con_power = (
        sum(p_.prod(i, p_.q) for i in p_.pro_var)
        - p_.prod(p_.li, p_.q)
        + p_.prod(p_.li_d, p_.q)
        - p_.sell(p_.power, p_.q)
        == 0
    )
    p_.con_charge = (
        p_.prod(p_.li, p_.q)
        - p_.prod(p_.li_d, p_.q)
        + p_.inv(p_.charge, p_.q - 1)
        - p_.inv(p_.charge, p_.q)
        == 0
    )
    p_.o = inf(sum(p_.ex_cap) + sum(p_.ex_vop) + sum(p_.ex_fop))
    p_.opt()
    return p_

In [3]:
p = p()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-10
Read MPS format model from file prog.mps
Reading time = 0.00 seconds
PROG: 40 rows, 31 columns, 81 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 40 rows, 31 columns and 81 nonzeros
Model fingerprint: 0x57d8ce44
Coefficient statistics:
  Matrix range     [5e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+02]
Presolve removed 30 rows and 20 columns
Presolve time: 0.00s
Presolved: 10 rows, 11 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9949000e+02   3.748738e+01   0.000000e+00      0s
       8    9.4200000e+05   0.000000e+00   0.000000e+00      0s

Solved in 8

In [11]:
p.pyomo()

pyomo is an optional dependency, pip install gana[all] to get optional dependencies


In [15]:
p.B 

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 -50.0,
 -100.0,
 -50.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [13]:
p.sol()

Solution for prog

---Objective Value(s)---



<IPython.core.display.Math object>


---Variable Value---



<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
p.con_sell.B

[-50.0, -100.0, -50.0]

In [11]:
len(p.o.funcs)

1

In [12]:
p.o.funcs.mis

1

In [13]:
p.o.funcs.variables

{0: [ex_cap_0,
  ex_cap_1,
  ex_cap_2,
  ex_vop_0,
  ex_vop_1,
  ex_vop_2,
  ex_fop_0,
  ex_fop_1,
  ex_fop_2]}

In [14]:
f = sum(p.ex_cap) + sum(p.ex_vop) + sum(p.ex_fop)

In [15]:
len(p.o.funcs)

1

In [16]:
len(f.X[0])

9

In [17]:
f.variables

{0: [ex_cap_0,
  ex_cap_1,
  ex_cap_2,
  ex_vop_0,
  ex_vop_1,
  ex_vop_2,
  ex_fop_0,
  ex_fop_1,
  ex_fop_2]}

In [18]:
for f in p.sets.constraint:
    print(f.variables)

{0: [cap_p_0], 1: [cap_p_1], 2: [cap_p_2]}
{0: [cap_s_0]}
{0: [sell_0], 1: [sell_1], 2: [sell_2]}
{0: [con_0], 1: [con_1], 2: [con_2]}
{0: [inv_0], 1: [inv_1], 2: [inv_2]}
{0: [prod_0], 1: [prod_1], 2: [prod_2], 3: [prod_3], 4: [prod_4], 5: [prod_5], 6: [prod_6], 7: [prod_7], 8: [prod_8]}
{0: [ex_cap_0], 1: [ex_cap_1], 2: [ex_cap_2]}
{0: [ex_fop_0], 1: [ex_fop_1], 2: [ex_fop_2]}
{0: [ex_vop_0], 1: [ex_vop_1], 2: [ex_vop_2]}
{0: [prod_0, prod_1, prod_2, ex_vop_0], 1: [prod_3, prod_4, prod_5, ex_vop_1], 2: [prod_6, prod_7, prod_8, ex_vop_2]}
{0: [cap_p_0], 1: [cap_p_1], 2: [cap_p_2]}
{0: [cap_s_0]}
{0: [con_0], 1: [con_1], 2: [con_2]}
{0: [sell_0], 1: [sell_1], 2: [sell_2]}
{0: [cap_p_0, prod_0], 1: [cap_p_0, prod_1], 2: [cap_p_0, prod_2]}
{0: [prod_3, cap_p_1], 1: [prod_4, cap_p_1], 2: [prod_5, cap_p_1], 3: [prod_6, cap_p_2], 4: [prod_7, cap_p_2], 5: [prod_8, cap_p_2]}
{0: [inv_0, cap_s_0], 1: [inv_1, cap_s_0], 2: [inv_2, cap_s_0]}
{0: [cap_p_0, ex_cap_0], 1: [cap_p_1, ex_cap_1], 2: [ca

In [19]:
p = Prg()
p.y = I(size=1)
p.q = I(size=3)
p.res_cons = I('solar')
p.res_dem = I('power')
p.res_stg = I('charge')
p.res = p.res_cons | p.res_dem | p.res_stg
p.pro_var = I('pv')
p.pro_cer = I('li', 'li_d')
p.pro = p.pro_var | p.pro_cer
p.dm_fac = P(p.power, p.q, _=[0.5, 1, 0.5])
# p.pv_fac = P(p.pv, p.q, _=[1, 0, 0.5])
p.demand = P(p.res_dem, p.q, _=[100] * 3)
p.capex = P(p.pro, p.y, _=[5000, 1000, 0])
p.fopex = P(p.pro, p.y, _=[500, 100, 0])
p.vopex = P(p.pro, p.y, _=[10, 50, 0])
p.cap_p = V(p.pro, p.y)

In [20]:
p.cap_s = V(p.res_stg, p.y)
p.sell = V(p.res_dem, p.q)
p.con = V(p.res_cons, p.q)
p.inv = V(p.res_stg, p.q)
p.prod = V(p.pro, p.q)
p.ex_cap = V(p.pro, p.y)
p.ex_fop = V(p.pro, p.y)
p.ex_vop = V(p.pro, p.y)

In [21]:
# p.stgu = T(p.charge, p.y, _=(0, 40))
p.pvfac = T(p.pv, p.q, _=[(0, 1), (0, 0.4), (0.4, 0.5)])

In [22]:
p.con_vopex = p.ex_vop(p.pro, p.y) == p.vopex(p.pro, p.y) * sum(
    p.prod(p.pro, q) for q in p.q
)
p.con_capmax = p.cap_p(p.pro, p.y) <= 200
p.con_capstg = p.cap_s(p.charge, p.y) <= 200
p.con_consmax = p.con(p.res_cons, p.q) <= 200

In [23]:
p.con_vopex.pprint(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [24]:
p.con_sell = p.sell(p.power, p.q) >= p.dm_fac(p.power, p.q) * p.demand(p.power, p.q)
# p.con_pv = p.prod(p.pv, p.q) <= p.pv_fac(p.pv, p.q) * p.cap_p(p.pv, p.y)
p.con_pv = p.prod(p.pv, p.q) <= 400 * p.pvfac(p.pv, p.q)

p.con_prod = p.prod(p.pro_cer, p.q) <= p.cap_p(p.pro_cer, p.y)
p.con_inv = p.inv(p.charge, p.q) <= p.cap_s(p.charge, p.y)
p.con_capex = p.ex_cap(p.pro, p.y) == p.capex(p.pro, p.y) * p.cap_p(p.pro, p.y)
p.con_fopex = p.ex_fop(p.pro, p.y) == p.fopex(p.pro, p.y) * p.cap_p(p.pro, p.y)
p.con_solar = p.prod(p.pv, p.q) == p.con(p.solar, p.q)
p.con_power = (
    sum(p.prod(i, p.q) for i in p.pro_var)
    - p.prod(p.li, p.q)
    + p.prod(p.li_d, p.q)
    - p.sell(p.power, p.q)
    == 0
)
p.con_charge = (
    p.prod(p.li, p.q)
    - p.prod(p.li_d, p.q)
    + p.inv(p.charge, p.q - 1)
    - p.inv(p.charge, p.q)
    == 0
)

In [25]:
p.con_sell.X, p.con_sell.A, p.con_sell.B

([[4, None, None], [None, 5, None], [None, None, 6]],
 [[-1.0, 0, 0], [0, -1.0, 0], [0, 0, -1.0]],
 [-50.0, -100.0, -50.0])

In [26]:
p.con_pv.X, p.con_pv.A, p.con_pv.F, p.con_pv.Z, p.con_pv.B

([[13, None, None], [None, 14, None], [None, None, 15]],
 [[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]],
 [[400.0, 0, 0], [0, 400.0, 0], [0, 0, 400.0]],
 [[0, None, None], [None, 1, None], [None, None, 2]],
 [0, 0, 0])

In [27]:
p.con_pv.F

[[400.0, 0, 0], [0, 400.0, 0], [0, 0, 400.0]]

In [28]:
p.con_fopex.pprint(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [29]:
p.pvfac.CRa, p.pvfac.CRb, p.pvfac.X

([[1.0, 0, 0],
  [-1.0, 0, 0],
  [0, 1.0, 0],
  [0, -1.0, 0],
  [0, 0, 1.0],
  [0, 0, -1.0]],
 [1, 0, 0.4, 0, 0.5, -0.4],
 [0, 1, 2])

In [31]:
p.CRa, p.CRb

([[1.0, 0, 0],
  [-1.0, 0, 0],
  [0, 1.0, 0],
  [0, -1.0, 0],
  [0, 0, 1.0],
  [0, 0, -1.0]],
 [1, 0, 0.4, 0, 0.5, -0.4])

In [31]:
p.con_fopex.X

[[0, None, None, 25, None, None],
 [None, 1, None, None, 26, None],
 [None, None, 2, None, None, 27]]

In [ ]:
p.F

In [ ]:
p.A

In [ ]:
p.inv(p.charge, p.q - 1)._

In [ ]:
p.con_sell.B

In [ ]:
p.con_inv.A

In [ ]:
p.con_prod.A

In [ ]:
p.con_pv.funcs.F

In [ ]:
p.vars()

In [ ]:
p.cons()

In [10]:
A = [[0] * len(p.vars()) for _ in range(len(p.cons()))]
B = [0] * len(p.cons())
F = [[0] * len(p.thetas) for _ in range(len(p.cons()))]

# A, B, F = ([0]*len(p.vars()) for _ in range(3))

In [ ]:
for c in p.sets.constraint:
    print(c.funcs)
    print(c.funcs.A)
    # print(c.funcs.vars)
    # print(c.funcs.pars)
    # print(c.funcs.funcs)
    # print(c.funcs.pvars)

In [ ]:
p.mps()

In [ ]:
for c in p.sets.constraint:
    print(c.funcs.one_, c.funcs.two_, c.funcs.elems)

In [ ]:
from gana.elements.func import Func

for n, c in enumerate(p.cons()):
    print(c.func)
    print(c.func.elems)

    # print(c.funcs.A)

In [ ]:
A

In [9]:
cs = p.sets.constraint

In [ ]:
cs[0].funcs._

In [ ]:
p.con_sell.funcs.B

In [ ]:
p.con_prod.funcs.A

In [ ]:
p.cons()

In [ ]:
p.constraints

In [ ]:
p.contvars()

In [ ]:
p.thetas

In [ ]:
p.pprint(True)

In [ ]:
p.con_prod.funcs.struct

In [ ]:
p.con_prod.two

In [ ]:
p.pprint()

In [ ]:
p.pvfac(p.pv, p.q).F

In [ ]:
p.con_pv.F

In [ ]:
p.con_sell.F

In [15]:
f = 400 * p.pvfac(p.pv, p.q)

In [ ]:
f.F

In [ ]:
p.con_sell.funcs.B

In [14]:
d = p.dm_fac(p.power, p.q) * p.demand(p.power, p.q)

In [ ]:
p.con_sell.funcs.A

In [ ]:
p.con_pv.F

In [ ]:
p.con_prod.funcs.A

In [ ]:
p.prod(p.pro_cer, p.q).A

In [ ]:
a = p.cap_p(p.pro_cer, p.y).A

In [ ]:
a

In [ ]:
[i for i in a for _ in range(3)]

In [ ]:
p.cap_p(p.pro_cer, p.y).A

In [ ]:
p.con_pv.pprint()

In [ ]:
p.con_sell.funcs.A

In [ ]:
p.con_pv.funcs.A

In [ ]:
p.con_pv.funcs.F

In [15]:
a = p.con_capex

In [ ]:
a.two

In [ ]:
for c in p.sets.constraint:
    c.pprint()
    # print(c.name, c.struct)
    # print(c.funcs.struct_)
    # print(c.funcs.rels_)
    # print()

In [ ]:
p.thetas

In [ ]:
p.con_capmax.two

In [14]:
f = p.sets.constraint[0].funcs

In [ ]:
f.struct

In [ ]:
f.signs

In [ ]:
f.vars

In [ ]:
for i in range(len(f.struct)):
    if i in f.vars:
        print(i, f.vars[i])
    if i in f.signs:
        print(i, f.signs[i])
    if i in f.pars:
        print(i, f.pars[i])

In [ ]:
for i in f.vars:
    print(i, f.vars[i])
    print(i - 1, f.signs[i - 1])
for i in f.pars:
    print(i, f.pars[i])
    print(i - 1, f.signs[i - 1])

In [ ]:
f.struct

In [ ]:
for i in f.vars:
    if f.signs[i - 1] == '×':
        print(f.pars[i - 2])

In [ ]:
f.array
# f.rel

In [ ]:
f.struct

In [ ]:
f.one

In [ ]:
f.two

In [ ]:
f.signs

In [ ]:
f.vars

In [ ]:
for c in p.sets.constraint:
    print(c.struct)
    # print(len(c.struct), c.struct)
    # print('array', c.funcs.array)
    # print(c.funcs.A())
    # print(c.funcs.signs, c.funcs.vars, c.funcs.pars, c.funcs.pvars)
    # print(c.funcs.elmo)

In [ ]:
x = p.sets.constraint[9].struct
if isinstance(x[-1], float) and x[-2] in ['+', '-']:
    # if there is a parameter in the end, that goes to the b matrix
    x = x[:-2]

# here you get a list of tuples (rel, variable)
x = list(zip(x[::2], x[1::2]))

In [ ]:
type(p.con_capmax.two)

In [ ]:
p.sets.constraint[9].struct

In [ ]:
p.sets.constraint[9].funcs.elmo

In [ ]:
p.struct

In [ ]:
p.sets.constraint[9].funcs.elmo

In [ ]:
for c in p.cons():
    print(c.struct)
    print(c.X(), c.A(), c.B(), c.T(), c.F())

In [ ]:
p.con_pv[0].pprint()

In [ ]:
for c in p.cons():
    print(c.F(), c.B())

In [ ]:
p.F()

In [ ]:
p.con_pv.funcs[1].vars

In [ ]:
p.thetas

In [ ]:
p.con_pv.pprint(True)

In [ ]:
p.con_pv[2].func.CRb()

In [33]:
a = [[], []]

In [36]:
a[0] = 4

In [ ]:
a

In [ ]:
for cons in p.cons():
    print(cons.F())

In [ ]:
p.pprint()

In [ ]:
for cons in p.cons():
    print(cons)
    cons.pprint()

In [ ]:
p.pprint()

In [ ]:
p.con_pv[0].F()

In [ ]:
p.thetas

In [ ]:
p.F()

In [ ]:
p.o = inf(sum(p.ex_cap) + sum(p.ex_vop) + sum(p.ex_fop))
p.opt()

In [ ]:
p.eqcons()[-2].func.A()

In [ ]:
p._A()[68][10]

In [ ]:
p._A()[69][11]

In [ ]:
p._A()[70][11]

In [ ]:
p.inv[0].n

In [ ]:
p.pprint()

In [ ]:
p.pprint(True)

In [ ]:
p.nncons()

In [ ]:
def p1():
    p1_ = Prg()
    p1_.i = I(size=2)
    p1_.j = I('a', 'b', 'c')
    p1_.x = V(p1_.i)
    p1_.y = V(p1_.j)
    p1_.c1 = sum(p1_.x) <= 12
    p1_.c2 = p1_.y(p1_.j) <= 20
    p1_.o1 = sup(40 * sum(p1_.x) + 30 * sum(p1_.y))
    return p1_


def p2():
    p2_ = Prg()
    p2_.k = I(size=2)
    p2_.z = V(p2_.k)
    p2_.c3 = sum(p2_.z) >= 4
    p2_.o2 = inf(3 * sum(p2_.z))
    return p2_

In [ ]:
p11 = p1()

In [ ]:
p11.A()

In [ ]:
p22 = p2()

In [ ]:
p22.A()

In [ ]:
p22.c3

In [ ]:
p = p11 + p22

In [ ]:
p11.sets.constraint

In [ ]:
p11.sets.nncons()

In [ ]:
p11.nncons()[0].two

In [ ]:
p.A()

In [ ]:
p.o1

In [ ]:
p.c1

In [ ]:
p.C()

In [ ]:
p.variables

In [ ]:
p.objectives

In [ ]:
p.o1

In [ ]:
def test_padd(p1, p2):
    p = p1 + p2
    assert p.A() == [
        [0, 0, 1.0, 0, 0, 0, 0],
        [0, 0, 0, 1.0, 0, 0, 0],
        [0, 0, 0, 0, 1.0, 0, 0],
        [1.0, 1.0, 0, 0, 0, 0, 0],
        [0, 0, 1.0, 0, 0, 0, 0],
        [0, 0, 0, 1.0, 0, 0, 0],
        [0, 0, 0, 0, 1.0, 0, 0],
        [-1.0, -1.0, 0, 0, 0, 0, 0],
        [-1.0, 0, 0, 0, 0, 0, 0],
        [0, -1.0, 0, 0, 0, 0, 0],
        [0, 0, -1.0, 0, 0, 0, 0],
        [0, 0, 0, -1.0, 0, 0, 0],
        [0, 0, 0, 0, -1.0, 0, 0],
        [0, 0, 0, 0, 0, -1.0, 0],
        [0, 0, 0, 0, 0, 0, -1.0],
        [-1.0, 0, 0, 0, 0, 0, 0],
        [0, -1.0, 0, 0, 0, 0, 0],
        [0, 0, -1.0, 0, 0, 0, 0],
        [0, 0, 0, -1.0, 0, 0, 0],
        [0, 0, 0, 0, -1.0, 0, 0],
        [-1.0, 0, 0, 0, 0, 0, 0],
        [0, -1.0, 0, 0, 0, 0, 0],
        [-1.0, 0, 0, 0, 0, 0, 0],
        [0, -1.0, 0, 0, 0, 0, 0],
        [0, 0, -1.0, 0, 0, 0, 0],
        [0, 0, 0, -1.0, 0, 0, 0],
        [0, 0, 0, 0, -1.0, 0, 0],
        [-1.0, 0, 0, 0, 0, 0, 0],
        [0, -1.0, 0, 0, 0, 0, 0],
    ]
    assert p.C() == [[-40.0, -40.0, -30.0, 30.0, 30.0, 0, 0], [3.0, 3.0, 0, 0, 0, 0, 0]]
    assert p.variables == [p.x_0, p.x_1, p.y_0, p.y_1, p.y_2, p.z_0, p.z_1]
    assert p.objectives == [p.o1, p.o2]

In [ ]:
p.ex_cap.index

In [ ]:
p.pprint()

In [ ]:
f = sum(p.ex_cap) + sum(p.ex_vop) + sum(p.ex_fop)

In [ ]:
f._

In [ ]:
p.ex_cap.index._

In [ ]:
a = (
    p.ex_cap(p.ex_cap.index._[0])
    + p.ex_cap(p.ex_cap.index._[1])
    + p.ex_cap(p.ex_cap.index._[2])
)

In [ ]:
sum(p.ex_cap)

In [ ]:
sum(p.ex_cap)

In [ ]:
p.con_charge.pprint(True)

In [ ]:
type(p.ex_cap)

In [ ]:
from operator import is_

is_(p.ex_vop(p.pro, p.y), p.ex_vop)

In [ ]:
type(p.ex_vop.index)

In [ ]:
p.ex_vop.index

In [ ]:
p.ex_vop(p.li, p.y[0]).index

In [ ]:
p.k = V()

In [ ]:
p.prod(p.pro, p.q[0])._

In [ ]:
p.ex_vop(p.pro, p.y[0])._

In [ ]:
from math import prod

a = prod((p.pro, p.y[0]))

In [ ]:
a._

In [ ]:
key = (p.li, p.y[0])

In [ ]:
k = 

In [ ]:
k = None
for x in key:
    k &= x

In [ ]:
k

In [ ]:
type(k)

In [ ]:
p.ex_vop.idx[k]

In [ ]:
type(p.li)

In [ ]:
type(p.y[0])

In [ ]:
p.ex_vop.idx

In [ ]:
type(p.ex_vop.index._[0])

In [ ]:
p.vopex(p.pro, p.y)

In [ ]:
p.ex_vop.index

In [ ]:
from math import prod

prod((p.pro, p.y))

In [ ]:
p.ex_vop.index._

In [ ]:
p.ex_vop._

In [ ]:
p.opt()

In [ ]:
p.sol()

In [ ]:
p = Prg()
p.i1 = I('a', 'b', 'c')
p.i2 = I(size=4)

In [ ]:
p.a._

In [ ]:
from energiapy.src.energiapy.gana.src.gana.elements.index import Idx

ii = p.i1 * Idx(4)

In [ ]:
ii._

In [ ]:
p.v1 = V(p.i1)
p.v2 = V(p.i2)
p.v12 = V(p.i1, p.i2)
p.v21 = V(p.i2, p.i1)
p.v14 = V(p.i1, 4)
p.v41 = V(4, p.i1)
p.va4 = V(p.a, 4)
p.v4a = V(4, p.a)

In [ ]:
p.v12(p.i1, 2)._

In [ ]:
p.i1.of

In [ ]:
p.v1.index

In [ ]:
list(p.va4.idx)[0]

In [ ]:
type(p.va4(p.a, 4))

In [ ]:
for i in p.i1:
    print(i, p.v1(i))
print()
for i in p.i2:
    print(i, p.v2(i))
print()

for i in p.i1:
    for j in p.i2:
        print((i, j), p.v12(i, j))
print()

for i in p.i2:
    for j in p.i1:
        print(p.v21(i, j))
print()

for i in p.i1:
    print(p.v14(i, 4))
print()

for j in p.i1:
    print(p.v41(4, j))
print()

print(p.va4(p.a, 4))
print()

print(p.v4a(4, p.a))

In [ ]:
list(p.v12.idx)[0]

In [ ]:
p.a.of

In [ ]:
p.v2.index._

In [ ]:
a = (1, 2, 3)

In [ ]:
p.i1.of

In [ ]:
def psmall():
    p_ = Prg()
    p_.i = I(size=2)
    p_.ri = I('r')
    p_.x = V(p_.i)
    p_.y = V(p_.ri)
    p_.c1 = p_.x(0) + p_.x(1) <= 12
    p_.c2 = 2 * p_.x(0) + p_.x(1) <= 16
    p_.c3 = 4 * p_.y(p_.r) == 40
    p_.o = sup(40 * p_.x(0) + 30 * p_.x(1) - p_.y(p_.r))
    # p_.opt()
    return p_

In [ ]:
p = Prg()

In [ ]:
p.i = I(size=2)
p.ri = I('r')

In [ ]:
p.ri.pprint()

In [ ]:
p.x = V(p.i)
p.y = V(p.ri)

In [ ]:
p.c1 = p.x(0) + p.x(1) <= 12
p.c2 = 2 * p.x(0) + p.x(1) <= 16
p.c3 = 4 * p.y(p.r) == 40
p.o = sup(40 * p.x(0) + 30 * p.x(1) - p.y(p.r))

In [ ]:
# a = p.oooo | p.ri

In [ ]:
p.oooo = I('r', 't', 'g')

In [ ]:
p.c1.X(), p.c1.A(), p.c1.B()

In [ ]:
p.r.parent

In [ ]:
p.NN()

In [ ]:
p = psmall()

In [ ]:
p.c3.pprint()

In [ ]:
p.mps()

In [ ]:
m = p.gurobi()

In [ ]:
p.opt()

In [ ]:
# p.pprint(True)

In [ ]:
p.lp()

In [ ]:
m = p.gurobi()

In [ ]:
# p.pyomo()

In [ ]:
p.opt()

In [ ]:
p.sol()

In [ ]:
p.opt()

In [ ]:
p.cap_p.sol()

In [ ]:
p.cap_p.idx

In [ ]:
p.cap_p(p.pro, p.y[0])

In [ ]:
p.sol()

In [ ]:
p.pprint(True)

In [ ]:
p.B()

In [ ]:
p.G()

In [ ]:
p.A()

In [ ]:
p.opt()

In [ ]:
p.cap_p.sol()

In [ ]:
p.sol()

In [ ]:
# p.o.sol(True)

In [ ]:
p_ = Prg()
p_.i = I(size=2)
p_.ri = I('r')
p_.x = V(p_.i)
p_.z = V(p_.ri)
p_.c1 = p_.x(0) + p_.x(1) <= 12
p_.c2 = 2 * p_.x(0) + p_.x(1) <= 16
p_.c3 = 4 * p_.z(p_.r) == 40
p_.ob = sup(40 * p_.x(0) + 30 * p_.x(1) - p_.z(p_.r))
# p_.opt()

In [ ]:
p1 = Prg()
p1.i = I(size=2)
p1.j = I('a', 'b', 'c')
p1.x = V(p1.i)
p1.y = V(p1.j)
p1.c1 = p1.x(p1.i[0]) + p1.x(p1.i[1]) <= 12
p1.c2 = p1.y(p1.j) <= 20
p1.o1 = sup(40 * sum(p1.x) + 30 * sum(p1.y))
# p1.opt()

In [ ]:
p.C()

In [ ]:
p2 = Prg()
p2.k = I(size=2)
p2.z = V(p2.k)
p2.c3 = p2.z(p2.k[0]) + p2.z(p2.k[1]) >= 4
# p2.o2 = inf(3 * sum(p2.z))
# p2.opt()

In [ ]:
p = p1 + p2

In [ ]:
p.variables

In [ ]:
for v in p.variables:
    print(v.n)

In [ ]:
p.pprint()

In [ ]:
p.A()

In [ ]:
p.opt()

In [ ]:
p.A()

In [ ]:
p.G()

In [ ]:
p.H()

In [ ]:
p.NN()

In [ ]:
p._A()

In [ ]:
p.C()

In [ ]:
p.objectives

In [ ]:
p.pprint()

In [ ]:
p1 = Prg()

In [ ]:
p1.i0 = I('x')
p1.i1 = I('x', 'y', 'z')
p1.i2 = I('p', 'q')
p1.i3 = I('x', 'y', 'z', 'p', 'q', 'y')
p1.i4 = I('x', 'y', 'z')
p1.i5 = I(size=3)
p1.i6 = I(size=800)

In [ ]:
# p1.i5 = p1.i0 * p1.i1
p1.v0 = V(p1.i0)
p1.v1 = V(p1.i1)
p1.v12 = V(p1.i1, nn=False)
p1.v2 = V(p1.i0, p1.i2, itg=True)
p1.v22 = V(p1.i0, p1.i2, nn=False)
p1.v32 = V(p1.i0, bnr=True)

In [ ]:
p1.p0 = P(p1.i0, p1.i5, _=[1, 2, 5])
p1.p1 = P(p1.i0, _=[4])
p1.p2 = P(p1.i0, p1.i5, _=[6, 8, 3])
p1.p3 = P(p1.i0, _=[True])
p1.p4 = P(p1.i0, p1.i5, _=[10, 10, 10])
p1.p5 = P(p1.i0, p1.i5, _=[4, 2, 7])
p1.p6 = P(p1.i1, _=[3, 4, 7])

In [ ]:
p1.f1 = p1.v22 + p1.v2

In [ ]:
p1.f1.pprint(True)

In [ ]:
p1.f2 = p1.v22 - p1.v2
p1.f3 = p1.v22 * p1.v2
p1.f4 = p1.v22 / p1.v2

p1.c1 = p1.v22 == p1.v2
p1.c2 = p1.v22 >= p1.v2
p1.c3 = p1.v22 <= p1.v2
p1.c4 = p1.v22 > p1.v2
p1.c5 = p1.v22 < p1.v2
p1.c6 = p1.v1 <= p1.p6

# p1.pprint(True)

In [ ]:
p1.pprint(True)

In [ ]:
p = Prg()

In [ ]:
p.t1 = I(size=1)
p.t2 = I(size=2)
p.t4 = I(size=4)

In [ ]:
p.c = V(p.t4)
p.s = V(p.t4)
p.p = V(p.t4)
p.bal = P(p.t4, _=[0, 0, 0, 0])
p.bub = P(p.t4, _=[12, 15, 18, 9])
p.blb = P(p.t4, _=[3, 3, 3, 3])
p.sub = P(p.t2, _=[15, 17])
p.slb = P(p.t2, _=[4, 8])
p.pub = P(p.t1, _=[50])
p.plb = P(p.t1, _=[10])
p.pcst = P(p.t4, _=[0.5] * 4)
p.bcst = P(p.t4, _=[2] * 4)
p.scst = P(p.t4, _=[3] * 4)
p.bcnv = P(p.t4, _=[3] * 4)
p.scnv = P(p.t4, _=[1] * 4)

In [ ]:
p.cons_bmax = p.c <= p.bub
p.cons_bmin = p.c + p.s + p.p >= p.blb
p.cons_smax1 = p.s(p.t4[0]) + 3 * p.s(p.t4[1]) <= p.sub(p.t2[0])
p.cons_smin1 = p.s(p.t4[0]) + p.s(p.t4[1]) >= p.slb(p.t2[0])
p.cons_smax2 = 3 + p.s(p.t4[2]) + p.s(p.t4[3]) <= p.sub(p.t2[1])
p.cons_smin2 = p.s(p.t4[2]) + p.s(p.t4[3]) >= p.slb(p.t2[0])
p.cons_bal = p.c + p.p - p.s == 0
p.cons_pmax = sum(p.p) <= sum(p.pub)

p.obj = inf(-3 * sum(p.s) + 0.5 * sum(p.p) + 2 * sum(p.c))


# p.pprint()
# p.pprint()

In [ ]:
p.t4.__dict__

In [ ]:
p.pprint(True)

In [ ]:
p.opt()

In [ ]:
p.sol()